In [1080]:
from qiskit import BasicAer, execute
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.tools.visualization import plot_histogram

import math
import fractions
import array
import numpy as np

In [1081]:
def shor (n, quantum=True):
    '''
    Inputs: integer N with unique factorization n = pq
    Outputs: p, q
    '''
    
    for i in range(2, n - 2):
        a = np.random.randint(2, n - 1)
        if is_coprime(a, n):
            # we are lucky!
            p = a
            q = int(n / a)
            if(p*q == n): return (p, q)
            
            if quantum:
                # use a quantum computer to find order m of U_Na
                m_list = qfind_order(a, n)
                if (m_list == None): return(0, 0)
            else:
                # use a classical algorithm to find order m
                m_list = find_order(a, n)

            for m in m_list:
                #No period found
                if m == 0: continue          

                if is_odd(m):
                    # invalid
                    continue
                else:
                    x = int(a ** (m / 2) % n)
                    if is_congruent(x, 1, n) or is_congruent(x, -1, n):
                        # invalid
                        continue
                    else:
                        # valid!
                        p = np.gcd(x - 1, n)
                        q = np.gcd(x + 1, n)
                        if(p*q == n): return (p, q)

    raise ValueError(f'{n} cannot be factored!')

In [1082]:
def is_coprime(a, b):
    '''
    Inputs: Integers a and b
    Outputs: true if a and b are coprime
    '''
    if np.gcd(a, b) == 1:
        return True
    else:
        return False
    
def is_odd(a):
    '''
    determines whether the number is odd
    Inputs: Integers a
    Outputs: true if a is odd
    '''
    if a & 0x1 == 1:
        return True
    else:
        return False
    
def is_congruent(a, b, n):
    '''
    determines whether a and b are congruent modulo n,
    Inputs: Integers a, b, n
    Outputs: true if congruence relation is satisfied
    '''
    if (a - b) % n == 0:
        return True
    else:
        return False

In [1083]:
def find_order(a, n):
    '''
    Brute force classical algorithm for order finding
    Inputs: Integers a and n
    Outputs: The period, or order r; the smallest (non-zero) integer such that:
    a**r mod n = 1
    '''
    m_list = []
    for m in range(2, int(n/2)):
        if(a**m % n == 1):
            m_list.append(m)
            return m_list

    return m_list

def qfind_order(a, n):
    '''
    Quantum algorithm for order finding
    Inputs: Integers a and n
    Outputs: The period, or order r; the smallest (non-zero) integer such that:
    a**r mod n = 1
    '''
    
    m = int(np.ceil(np.log2(n)))
    a = 4
    n_smoz = 15
    if (n==n_smoz):
        #Smoz et al. (2016) algorithm for n = 15
        qr = QuantumRegister(m + 1, 'qr')
        cr = ClassicalRegister(m + 1, 'cr')

        qc = QuantumCircuit(qr, cr)
        print(f'Number of qubits = {m+1}')
        circuit_aperiod15(qc, qr, cr, a)

    else:
       # Beauregard (2003) algorithm for general n
        #Create quantum and classical registers """
        # auxilliary quantum register used in addition and multiplication
        aux = QuantumRegister(m+2)
        
        #single qubit where the sequential QFT is performed
        up_reg = QuantumRegister(1)
        
        #quantum register where the multiplications are made
        down_reg = QuantumRegister(m)
        
        #classical register where the measured values of the sequential QFT are stored
        up_classic = ClassicalRegister(2*m)
        
        #classical bit used to reset the state of the top qubit to 0 if the previous measurement was 1
        c_aux = ClassicalRegister(1)

        #Create Quantum Circuit
        qc = QuantumCircuit(down_reg , up_reg , aux, up_classic, c_aux)
        
        print(f'Number of qubits = {2*m+3}')
        
        shor_sequential_QFT(qc, m, down_reg , up_reg , aux, up_classic, c_aux, a, n)
        

    #Simulate the created Quantum Circuit """
#    print(f'Circuit depth = {qc.depth()}')
    print(f'Number of gates = {sum(qc.count_ops().values())}')
    number_shots = 200
    simulation = execute(qc, backend=BasicAer.get_backend('qasm_simulator'),shots=number_shots)
    """ to run on IBM, use backend=IBMQ.get_backend('ibmq_qasm_simulator') in execute() function """
    """ to run locally, use backend=BasicAer.get_backend('qasm_simulator') in execute() function """

    """ Get the results of the simulation in proper structure """
    sim_result=simulation.result()
    counts_result = sim_result.get_counts(qc)
    output_list = list(sim_result.get_counts().keys())
    
    if(n == n_smoz):
        m_list = []
        for o in output_list:
            m_list.append(int(o, 2))
    
        return m_list
    
    #General case
    else:
        for o in output_list:
            output_desired = o.split(" ")[1]
            x_value = int(output_desired, 2)
            # Get the factors using the x value obtained  
            success = get_factors(int(x_value),int(2*m),int(n),int(a))
            if(success): return None
            
        return None

In [1084]:
# Adapted from
# https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/algorithms/shor_algorithm.ipynb
# Impelmentation of Monz et al. (2016) algorithm for N = 15
# Monz T, Nigg D, Martinez EA, Brandl MF, Schindler P, Rines R, Wang SX, Chuang IL, Blatt R.
# 2016. Realization of a scalable Shor algorithm. Science, 351(6277), pp.1068-1070.

# qc = quantum circuit, qr = quantum register, cr = classical register, a = 2, 7, 8, 11 or 13
def circuit_aperiod15(qc, qr, cr, a):
    if a == 11:
        circuit_11period15(qc, qr, cr)
        return
    
    # Initialize q[0] to |1> 
    qc.x(qr[0])

    # Apply a**4 mod 15
    qc.h(qr[4])
    #   controlled identity on the remaining 4 qubits, which is equivalent to doing nothing
    qc.h(qr[4])
    #   measure
    qc.measure(qr[4],cr[0])
    #   reinitialise q[4] to |0>
    qc.reset(qr[4])

    # Apply a**2 mod 15
    qc.h(qr[4])
    #   controlled unitary
    qc.cx(qr[4],qr[2])
    qc.cx(qr[4],qr[0])
    #   feed forward
    qc.p(np.pi/2.,qr[4]).c_if(cr, 1)
    qc.h(qr[4])
    #   measure
    qc.measure(qr[4],cr[1])
    #   reinitialise q[4] to |0>
    qc.reset(qr[4])

    # Apply a mod 15
    qc.h(qr[4])
    #   controlled unitary.
    circuit_amod15(qc,qr,cr,a)
    #   feed forward
    qc.p(3.*np.pi/4.,qr[4]).c_if(cr, 3)
    qc.p(np.pi/2.,qr[4]).c_if(cr, 2)
    qc.p(np.pi/4.,qr[4]).c_if(cr, 1)
    qc.h(qr[4])
    #   measure
    qc.measure(qr[4],cr[2])

# qc = quantum circuit, qr = quantum register, cr = classical register, a = 2, 7, 8, 11 or 13
def circuit_amod15(qc,qr,cr,a):
    if a == 2:
        qc.cswap(qr[4],qr[3],qr[2])
        qc.cswap(qr[4],qr[2],qr[1])
        qc.cswap(qr[4],qr[1],qr[0])
    elif a == 7:
        qc.cswap(qr[4],qr[1],qr[0])
        qc.cswap(qr[4],qr[2],qr[1])
        qc.cswap(qr[4],qr[3],qr[2])
        qc.cx(qr[4],qr[3])
        qc.cx(qr[4],qr[2])
        qc.cx(qr[4],qr[1])
        qc.cx(qr[4],qr[0])
    elif a == 8:
        qc.cswap(qr[4],qr[1],qr[0])
        qc.cswap(qr[4],qr[2],qr[1])
        qc.cswap(qr[4],qr[3],qr[2])
    elif a == 11: # this is included for completeness
        qc.cswap(qr[4],qr[2],qr[0])
        qc.cswap(qr[4],qr[3],qr[1])
        qc.cx(qr[4],qr[3])
        qc.cx(qr[4],qr[2])
        qc.cx(qr[4],qr[1])
        qc.cx(qr[4],qr[0])
    elif a == 13:
        qc.cswap(qr[4],qr[3],qr[2])
        qc.cswap(qr[4],qr[2],qr[1])
        qc.cswap(qr[4],qr[1],qr[0])
        qc.cx(qr[4],qr[3])
        qc.cx(qr[4],qr[2])
        qc.cx(qr[4],qr[1])
        qc.cx(qr[4],qr[0])


def circuit_11period15(qc,qr,cr):
    # Initialize q[0] to |1> 
    qc.x(qr[0])

    # Apply a**4 mod 15
    qc.h(qr[4])
    #   controlled identity on the remaining 4 qubits, which is equivalent to doing nothing
    qc.h(qr[4])
    #   measure
    qc.measure(qr[4],cr[0])
    #   reinitialise q[4] to |0>
    qc.reset(qr[4])

    # Apply a**2 mod 15
    qc.h(qr[4])
    #   controlled identity on the remaining 4 qubits, which is equivalent to doing nothing
    #   feed forward
    qc.p(np.pi/2.,qr[4]).c_if(cr, 1)
    qc.h(qr[4])
    #   measure
    qc.measure(qr[4],cr[1])
    #   reinitialise q[4] to |0>
    qc.reset(qr[4])

    # Apply 11 mod 15
    qc.h(qr[4])
    #   controlled unitary.
    qc.cx(qr[4],qr[3])
    qc.cx(qr[4],qr[1])
    #   feed forward
    qc.p(3.*np.pi/4.,qr[4]).c_if(cr, 3)
    qc.p(np.pi/2.,qr[4]).c_if(cr, 2)
    qc.p(np.pi/4.,qr[4]).c_if(cr, 1)
    qc.h(qr[4])
    #   measure
    qc.measure(qr[4],cr[2])

In [1085]:
#adapted from
#https://github.com/ttlion/ShorAlgQiskit/blob/master/Shor_Sequential_QFT.py
#Stephane Beauregard (2003), Circuit for Shors algorithm using 2n+3 qubits, Quantum Information
#and Computation, Vol. 3, No. 2 (2003) pp. 175-185. Also on quant-ph/0205095

# Note notation changes from the calling routine
# Calling routine        In this cell
#   qc                     circuit
#   n                      N
#   m                      n

def shor_sequential_QFT(circuit, n, down_reg , up_reg , aux, up_classic, c_aux, a, N):

    #Initialize down register to 1
    circuit.x(down_reg[0])

    #Cycle to create the Sequential QFT, measuring qubits and applying the right gates according to measurements
    for i in range(0, 2*n):
        #reset the top qubit to 0 if the previous measurement was 1
        circuit.x(up_reg).c_if(c_aux, 1)
        circuit.h(up_reg)
        cMULTmodN(circuit, up_reg[0], down_reg, aux, a**(2**(2*n-1-i)), N, n)
        #cycle through all possible values of the classical register and apply the corresponding conditional phase shift
        for j in range(0, 2**i):
            #the phase shift is applied if the value of the classical register matches j exactly
            circuit.p(getAngle(j, i), up_reg[0]).c_if(up_classic, j)
        circuit.h(up_reg)
        circuit.measure(up_reg[0], up_classic[i])
        circuit.measure(up_reg[0], c_aux[0])
    

#Circuit that implements single controlled modular multiplication by a
def cMULTmodN(circuit, ctl, q, aux, a, N, n):
    create_QFT(circuit,aux,n+1,0)
    for i in range(0, n):
        ccphiADDmodN(circuit, aux, q[i], ctl, aux[n+1], (2**i)*a % N, N, n+1)
    create_inverse_QFT(circuit, aux, n+1, 0)

    for i in range(0, n):
        circuit.cswap(ctl,q[i],aux[i])

    a_inv = modinv(a, N)
    create_QFT(circuit, aux, n+1, 0)
    i = n-1
    while i >= 0:
        ccphiADDmodN_inv(circuit, aux, q[i], ctl, aux[n+1], math.pow(2,i)*a_inv % N, N, n+1)
        i -= 1
    create_inverse_QFT(circuit, aux, n+1, 0)


#Function to create the QFT
def create_QFT(circuit,up_reg,n,with_swaps):
    i=n-1
    """ Apply the H gates and Cphases
    The Cphases with |angle| < threshold are not created because they do 
    nothing. The threshold is put as being 0 so all CPhases are created,
    but the clause is there so if wanted just need to change the 0 of the
    if-clause to the desired value
    """
    while i>=0:
        circuit.h(up_reg[i])        
        j=i-1  
        while j>=0:
            if (np.pi)/(pow(2,(i-j))) > 0:
                circuit.cp( (np.pi)/(pow(2,(i-j))) , up_reg[i] , up_reg[j] )
                j=j-1   
        i=i-1  

    """ If specified, apply the Swaps at the end """
    if with_swaps==1:
        i=0
        while i < ((n-1)/2):
            circuit.swap(up_reg[i], up_reg[n-1-i])
            i=i+1

#Function to create inverse QFT """
def create_inverse_QFT(circuit,up_reg,n,with_swaps):
    """ If specified, apply the Swaps at the beggining"""
    if with_swaps==1:
        i=0
        while i < ((n-1)/2):
            circuit.swap(up_reg[i], up_reg[n-1-i])
            i=i+1
    
    """ Apply the H gates and Cphases"""
    """ The Cphases with |angle| < threshold are not created because they do 
    nothing. The threshold is put as being 0 so all CPhases are created,
    but the clause is there so if wanted just need to change the 0 of the
    if-clause to the desired value """
    i=0
    while i<n:
        circuit.h(up_reg[i])
        if i != n-1:
            j=i+1
            y=i
            while y>=0:
                 if (np.pi)/(pow(2,(j-y))) > 0:
                    circuit.cp( - (np.pi)/(pow(2,(j-y))) , up_reg[j] , up_reg[y] )
                    y=y-1   
        i=i+1

#Function that calculates the angle of a phase shift in the sequential QFT based on the binary digits of a."""
#a represents a possile value of the classical register"""
def getAngle(a, N):
    """convert the number a to a binary string with length N"""
    s=bin(int(a))[2:].zfill(N) 
    angle = 0
    for i in range(0, N):
        """if the digit is 1, add the corresponding value to the angle"""
        if s[N-1-i] == '1': 
            angle += math.pow(2, -(N-i))
    angle *= np.pi
    return angle

#Function that calculates the array of angles to be used in the addition in Fourier Space"""
def getAngles(a,N):
    s=bin(int(a))[2:].zfill(N) 
    angles=np.zeros([N])
    for i in range(0, N):
        for j in range(i,N):
            if s[j]=='1':
                angles[N-i-1]+=math.pow(2, -(j-i))
        angles[N-i-1]*=np.pi
    return angles

#Creation of a doubly controlled phase gate"""
def ccphase(circuit, angle, ctl1, ctl2, tgt):
    circuit.cp(angle/2,ctl1,tgt)
    circuit.cx(ctl2,ctl1)
    circuit.cp(-angle/2,ctl1,tgt)
    circuit.cx(ctl2,ctl1)
    circuit.cp(angle/2,ctl2,tgt)

#Creation of the circuit that performs addition by a in Fourier Space"""
#Can also be used for subtraction by setting the parameter inv to a value different from 0"""
def phiADD(circuit, q, a, N, inv):
    angle=getAngles(a,N)
    for i in range(0,N):
        if inv==0:
            circuit.p(angle[i],q[i])
            """addition"""
        else:
            circuit.p(-angle[i],q[i])
            """subtraction"""

#Single controlled version of the phiADD circuit"""
def cphiADD(circuit, q, ctl, a, n, inv):
    angle=getAngles(a,n)
    for i in range(0,n):
        if inv==0:
            circuit.cp(angle[i],ctl,q[i])
        else:
            circuit.cp(-angle[i],ctl,q[i])
        
#Doubly controlled version of the phiADD circuit"""
def ccphiADD(circuit,q,ctl1,ctl2,a,n,inv):
    angle=getAngles(a,n)
    for i in range(0,n):
        if inv==0:
            ccphase(circuit,angle[i],ctl1,ctl2,q[i])
        else:
            ccphase(circuit,-angle[i],ctl1,ctl2,q[i])
        
#Circuit that implements doubly controlled modular addition by a"""
def ccphiADDmodN(circuit, q, ctl1, ctl2, aux, a, N, n):
    ccphiADD(circuit, q, ctl1, ctl2, a, n, 0)
    phiADD(circuit, q, N, n, 1)
    create_inverse_QFT(circuit, q, n, 0)
    circuit.cx(q[n-1],aux)
    create_QFT(circuit,q,n,0)
    cphiADD(circuit, q, aux, N, n, 0)
    
    ccphiADD(circuit, q, ctl1, ctl2, a, n, 1)
    create_inverse_QFT(circuit, q, n, 0)
    circuit.x(q[n-1])
    circuit.cx(q[n-1], aux)
    circuit.x(q[n-1])
    create_QFT(circuit,q,n,0)
    ccphiADD(circuit, q, ctl1, ctl2, a, n, 0)

#Circuit that implements the inverse of doubly controlled modular addition by a"""
def ccphiADDmodN_inv(circuit, q, ctl1, ctl2, aux, a, N, n):
    ccphiADD(circuit, q, ctl1, ctl2, a, n, 1)
    create_inverse_QFT(circuit, q, n, 0)
    circuit.x(q[n-1])
    circuit.cx(q[n-1],aux)
    circuit.x(q[n-1])
    create_QFT(circuit, q, n, 0)
    ccphiADD(circuit, q, ctl1, ctl2, a, n, 0)
    cphiADD(circuit, q, aux, N, n, 1)
    create_inverse_QFT(circuit, q, n, 0)
    circuit.cx(q[n-1], aux)
    create_QFT(circuit, q, n, 0)
    phiADD(circuit, q, N, n, 0)
    ccphiADD(circuit, q, ctl1, ctl2, a, n, 1)
    
# Function to apply the continued fractions to find r and the gcd to find the desired factors
def get_factors(x_value,t_upper,N,a):

    if x_value<=0:
        print('x_value is <= 0, there are no continued fractions\n')
        return False

    print('Running continued fractions for this case\n')

    """ Calculate T and x/T """
    T = pow(2,t_upper)

    x_over_T = x_value/T

    """ Cycle in which each iteration corresponds to putting one more term in the
    calculation of the Continued Fraction (CF) of x/T """

    """ Initialize the first values according to CF rule """
    i=0
    b = array.array('i')
    t = array.array('f')

    b.append(math.floor(x_over_T))
    t.append(x_over_T - b[i])

    while i>=0:

        """From the 2nd iteration onwards, calculate the new terms of the CF based
        on the previous terms as the rule suggests"""

        if i>0:
            b.append( math.floor( 1 / (t[i-1]) ) ) 
            t.append( ( 1 / (t[i-1]) ) - b[i] )

        """ Calculate the CF using the known terms """

        aux = 0
        j=i
        while j>0:    
            aux = 1 / ( b[j] + aux )      
            j = j-1
        
        aux = aux + b[0]

        """Get the denominator from the value obtained"""
        frac = fractions.Fraction(aux).limit_denominator()
        den=frac.denominator

        print('Approximation number {0} of continued fractions:'.format(i+1))
        print("Numerator:{0} \t\t Denominator: {1}\n".format(frac.numerator,frac.denominator))

        """ Increment i for next iteration """
        i=i+1

        if (den%2) == 1:
            if i>=15:
                print('Returning because have already done too much tries')
                return False
            print('Odd denominator, will try next iteration of continued fractions\n')
            continue
    
        """ If denominator even, try to get factors of N """

        """ Get the exponential a^(r/2) """

        exponential = 0

        if den<1000:
            exponential=pow(a , (den/2))
        
        """ Check if the value is too big or not """
        if math.isinf(exponential)==1 or exponential>1000000000:
            print('Denominator of continued fraction is too big!\n')
            aux_out = input('Input number 1 if you want to continue searching, other if you do not: ')
            if aux_out != '1':
                return False
            else:
                continue

        """If the value is not to big (infinity), then get the right values and
        do the proper gcd()"""

        putting_plus = int(exponential + 1)

        putting_minus = int(exponential - 1)
    
        one_factor = math.gcd(putting_plus,N)
        other_factor = math.gcd(putting_minus,N)
    
        """ Check if the factors found are trivial factors or are the desired
        factors """

        if one_factor==1 or one_factor==N or other_factor==1 or other_factor==N:
            print('Found just trivial factors, not good enough\n')
            """ Check if the number has already been found, use i-1 because i was already incremented """
            if t[i-1]==0:
                print('The continued fractions found exactly x_final/(2^(2n)) , leaving funtion\n')
                return False
            if i<15:
                return False
#                aux_out = input('Input number 1 if you want to continue searching, other if you do not: ')
#                if aux_out != '1':
#                    return False       
            else:
                """ Return if already too much tries and numbers are huge """ 
                print('Returning because have already done too many tries\n')
                return False         
        else:
            print('The factors of {0} are {1} and {2}\n'.format(N,one_factor,other_factor))
            print('Found the desired factors!\n')
            return True


#Functions that calculate the modular inverse using Euclid's algorithm"""
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)
def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

In [1086]:
#tests
a = 22
b = 15
print(f'{a} and {b} are coprime? {is_coprime(a, b)}')

a = 3
print(f'{a} is odd? {is_odd(a)}')

a = -8
b = 7
n = 5
print(f'{a} and {b} are congruent modulo {n}? {is_congruent(a, b, n)}')

a = 3
n = 35
print(f'Classical: Order of {a, n} = {find_order(a, n)}')

22 and 15 are coprime? True
3 is odd? True
-8 and 7 are congruent modulo 5? True
Classical: Order of (3, 35) = [12]


In [1087]:
%%time
n = 91
a, b = shor(n)
if a != 0:
    print(f'Prime factors of {n} are {a, b}')

Number of qubits = 17
Number of gates = 74232
x_value is <= 0, there are no continued fractions

Running continued fractions for this case

Approximation number 1 of continued fractions:
Numerator:0 		 Denominator: 1

Odd denominator, will try next iteration of continued fractions

Approximation number 2 of continued fractions:
Numerator:1 		 Denominator: 3

Odd denominator, will try next iteration of continued fractions

Approximation number 3 of continued fractions:
Numerator:5461 		 Denominator: 16384

Found just trivial factors, not good enough

Running continued fractions for this case

Approximation number 1 of continued fractions:
Numerator:0 		 Denominator: 1

Odd denominator, will try next iteration of continued fractions

Approximation number 2 of continued fractions:
Numerator:1 		 Denominator: 2

Found just trivial factors, not good enough

The continued fractions found exactly x_final/(2^(2n)) , leaving funtion

Running continued fractions for this case

Approximation numb